In [ ]:
REFRESH_TOKEN = 'ZT9RCsKBcwHaLCPulKmDFXWPxpwfoYtAdYunn5qgeVWnpzYBEkw5qXzFraNgPIAS'
api_url = 'https://api.chartmetric.com/api/token'
data = {'refreshtoken' : REFRESH_TOKEN}
json = {'Content-Type' : 'application/json'}

import numpy as np


In [ ]:
import requests
import json
import pandas as pd
import time

In [ ]:
df = pd.read_csv('spotify_playlist_tracks.csv', index_col=0)

cm_artist_ids = list(df.CM_Artist_ID)
cm_track_ids = list(df.CM_Track_ID)

In [ ]:
track_ids

In [ ]:
track_ids = pd.Series(cm_track_ids).dropna()
cm_track_ids = [int(ID) for ID in cm_track_ids]

In [ ]:
track_ids = list(track_ids)

In [ ]:
cm_track_ids = [int(ID) for ID in cm_track_ids]

In [ ]:
one_track_id = cm_track_ids[0]

In [ ]:
#create function to retrieve access token
def get_api_token(url, data, json):

    response = requests.post(url=api_url, data={'refreshtoken' : REFRESH_TOKEN}, 
                             json={'Content-Type' : 'application/json'})
    results = response.json()
    api_token = results['token']
    return api_token
api_token = get_api_token(api_url, data, json)  

In [ ]:
#create a function to call meta data for one track

headers={'Authorization' : 'Bearer {}'.format(api_token)}

def get_track_metadata(headers, one_track_id):

        
    response = requests.get(url='https://api.chartmetric.com/api/track/{}'.format(one_track_id),
                            headers=headers
                                )
    if response.status_code == 200:

        data = response.json()
        track = data['obj']
    else:
        pass
    return track

        
    
    

In [ ]:
track = get_track_metadata(headers, one_track_id)


In [ ]:
def parse_track_metadata(track):
    try:

        value_tuple = (track['id'], track['name'], track['artists'][0]['name'], track['artists'][0]['gender'], 
                            track['artists'][0]['date_of_birth'], 
                            track['albums'][0]['name'], track['albums'][0]['release_date'], 
                            track['albums'][0]['label'], track['cm_statistics']['num_sp_playlists'],
                            track['cm_statistics']['sp_playlist_total_reach'], track['cm_audio_features']['key'],
                            track['cm_audio_features']['danceability'], track['cm_audio_features']['energy'],
                            track['cm_audio_features']['acousticness'], 
                            track['cm_audio_features']['instrumentalness'], track['cm_audio_features']['liveness'],
                            track['cm_audio_features']['valence'], track['cm_audio_features']['tempo'], 
                            track['cm_audio_features']['loudness'])
    except:
        value_tuple = (track['id'], track['name'], track['artists'][0]['name'], track['artists'][0]['gender'], 
                            track['artists'][0]['date_of_birth'], 
                            track['albums'][0]['name'], track['albums'][0]['release_date'], 
                            track['albums'][0]['label'], ' n',
                            'nan', track['cm_audio_features']['key'],
                            track['cm_audio_features']['danceability'], track['cm_audio_features']['energy'],
                            track['cm_audio_features']['acousticness'], 
                            track['cm_audio_features']['instrumentalness'], track['cm_audio_features']['liveness'],
                            track['cm_audio_features']['valence'], track['cm_audio_features']['tempo'], 
                            track['cm_audio_features']['loudness'])
    return value_tuple

In [ ]:
# parse_track_metadata(track)

In [ ]:
#create key tuple
key_tuple = ('CM_ID', 'Track_Name', 'Artist_Name', 'Gender', 'DOB', 'Album_Name', 'Release_Date', 
                                     'Label', 'Num_Sp_Playlists', 'Playlist_Total_Reach', 'Key', 
                                     'danceability', 'energy', 'acoustiness', 'instrumentalness', 'liveness',
                                     'valence', 'tempo', 'loudness')
  
#create a dictionary with keys as column names
track_dict = {k:[] for k in  key_tuple }
track_dict

In [ ]:
#put all the functions in a for loop to grab metadata for all 8000 tracks
count = 0
for one_track_id in track_ids:
    count += 1

    try:
        track = get_track_metadata(headers, one_track_id)
    except:
        print('No track found', count)
    finally:
        pass
    
    
        value_tuple = parse_track_metadata(track)
    

        for i in range(len(track_dict)):

            list(track_dict.values())[i].append(value_tuple[i])


        with open('track_metadata.json', 'w') as json_file:
            json.dump(track_dict, json_file)

In [ ]:
list(track_dict.values())[0]

In [ ]:
df = pd.read_json('track_metadata.json')

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
# for key, value in track_dict.items():
#     print(type(value))